<a href="https://colab.research.google.com/github/kidusalamrewsqralex-droid/my_model/blob/main/Smart_Plant_Disease_Detection_System(SPDDS).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q tensorflow tensorflow-datasets

In [ ]:
!pip install tensorflow.keras

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
dataset,info=tfds.load('plant_village',with_info=True,as_supervised=True)
teain_ds_full=dataset['train']
print('Number of classes: ',info.features['label'].num_classes)
print('Class names: ',info.features['label'].names)

In [ ]:
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

def preprocess(image, label):
    image = tf.image.resize(image, IMG_SIZE)
    image = image / 255.0
    return image, label

train_ds_full = dataset['train'].map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)

total_examples = info.splits["train"].num_examples
train_size = int(0.8 * total_examples)
val_size = total_examples - train_size

train_ds = train_ds_full.take(train_size).shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_ds = train_ds_full.skip(train_size).take(val_size).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),

    Dense(38, activation='softmax')
])

In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    callbacks=[early_stop]
)

In [ ]:
model.save("SPDDS.keras")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
model.save('/content/drive/MyDrive/my_model.keras')

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
import numpy as np
model = load_model('/content/drive/MyDrive/my_model.keras')
img = image.load_img('thumb.jpg', target_size=(128, 128),color_mode='rgb')
img_array = image.img_to_array(img)
img_array = img_array / 255.0
img_array = np.expand_dims(img_array, axis=0)
prediction = model.predict(img_array)
predicted_class=np.argmax(prediction,axis=1)[0]
class_names=info.features['label'].names
print('prediction: ',class_names[predicted_class])
print("Prediction probabilities:", prediction)